In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [16]:
html = requests.get('https://sklep.ariadna.com.pl/mulina/145-mulina-8-m.html').text
soup = BeautifulSoup(html, 'html.parser')

In [17]:
def read_data():
    for color_obj in soup.find_all(attrs={'class': 'color_pick'}):
        number = color_obj.get('name')

        rgb_match = re.match(r'.*(#[\da-fA-F]{6})', color_obj.get('style'))
        rgb = rgb_match.group(1).lower() if rgb_match else None

        yield (number, rgb)


ariadna_colors_df = pd.DataFrame(
    [color_data for color_data in read_data()],
    columns=['number', 'rgb']
).set_index('number')

In [18]:
ariadna_colors_df

,rgb
number,
1500,#ffffff
1501,#f2efc9
1502,#ffff9f
1503,#ffff62
1504,#fafa00
...,...
1782A,#dec2a2
1782B,#fcf3e8
1794A,#807568


In [19]:
ariadna_df = pd.read_csv('../../data/ariadna.csv', index_col='number')
ariadna_df

,pl,name
number,,
1500,biały,white
1501,cytrynowy najjaśniejszy,lemon ultra light
1502,cytrynowy jasny,lemon light
1503,cytrynowy,lemon
1504,cytrynowy ciemny,lemon dark
...,...,...
1782A,kawowobeżowy średni,mocha beige medium
1782B,beżowobrązowy najjaśniejszy,beige brown ultra light
1794A,brązowy korzenny ciemny,driftwood dark


In [26]:
ariadna_df = ariadna_df.join(ariadna_colors_df)
ariadna_df

,pl,name,rgb
number,,,
1500,biały,white,#ffffff
1501,cytrynowy najjaśniejszy,lemon ultra light,#f2efc9
1502,cytrynowy jasny,lemon light,#ffff9f
1503,cytrynowy,lemon,#ffff62
1504,cytrynowy ciemny,lemon dark,#fafa00
...,...,...,...
1782A,kawowobeżowy średni,mocha beige medium,#dec2a2
1782B,beżowobrązowy najjaśniejszy,beige brown ultra light,#fcf3e8
1794A,brązowy korzenny ciemny,driftwood dark,#807568


In [27]:
ariadna_df.loc[ariadna_df['rgb'] == None]

,pl,name,rgb
number,,,


In [28]:
ariadna_df.to_csv('../../data/ariadna.csv')